<a href="https://colab.research.google.com/github/feist000/Project/blob/master/%EA%B3%A0%EC%96%91%EC%8B%9C%20%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0%20%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%85%98/2_%EA%B3%A0%EC%96%91%EC%8B%9C_%EB%8C%80%EC%A4%91%EA%B5%90%ED%86%B5_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 지하철 데이터를 기반으로 지하철역별 유동인구 데이터 추출하기

In [ ]:
import pandas as pd
import pathlib

import numpy as np

from tqdm.notebook import tqdm


from geoband import API

In [ ]:
# 파일 불러오기
input_path = pathlib.Path('./input')
if not input_path.is_dir():
    input_path.mkdir()    
        
API.GetCompasData('SBJ_2007_001', '1', input_path.joinpath('01.운영이력.csv'))
API.GetCompasData('SBJ_2007_001', '2', input_path.joinpath('02.자전거스테이션.csv'))
API.GetCompasData('SBJ_2007_001', '19',input_path.joinpath('19.전철역_공간정보.csv'))
API.GetCompasData('SBJ_2007_001', '29',input_path.joinpath('29.지하철 역별 이용객수.csv'))
API.GetCompasData('SBJ_2007_001', '20', input_path.joinpath('20.고양시_버스정류소.csv'))
API.GetCompasData('SBJ_2007_001', '21', input_path.joinpath('21.버스_정류장별_승하차_정보.csv'))


bicycle=pd.read_csv(input_path.joinpath('01.운영이력.csv'))
station_info = pd.read_csv(input_path.joinpath('02.자전거스테이션.csv'))
station_location = pd.read_csv(input_path.joinpath("19.전철역_공간정보.csv"))
station_data = pd.read_csv(input_path.joinpath("29.지하철 역별 이용객수.csv"))

# 코드 설명
## - 19. 전철역_공간정보
- rail_nm    해당 역의 호선 명입니다.
- station_nm    역 명입니다.
- lot_num_addr    해당 역의 지번 주소입니다.
- road_nm_addr    해당 역의 도로명 주소입니다.
- X    지하철역의 위도 입니다.
- Y    지하철역의 경도 입니다.

## 29. 지하철 역별 이용객수
- 호선명    해당 역의 호선 명입니다.
-  역명    역 명입니다.
- 행정구역    해당 역의 행정구역(동) 입니다.
- 년월구분    해당 라인 데이터의 년월구분 입니다.
- 승하차구분    해당 라인 데이터의 승하차구분 입니다.
- 월요일_합계    해당 역의 승하차 구분별 월요일 합계입니다.
- 화요일_합계    해당 역의 승하차 구분별 화요일 합계입니다.
- 수요일_합계    해당 역의 승하차 구분별 수요일 합계입니다.
- 목요일_합계    해당 역의 승하차 구분별 목요일 합계입니다.
- 금요일_합계    해당 역의 승하차 구분별 금요일 합계입니다.
- 토요일_합계    해당 역의 승하차 구분별 토요일 합계입니다.
- 일요일_합계    해당 역의 승하차 구분별 일요일 합계입니다.
- 모든요일_합계    해당 역의 승하차 구분별 모든요일 합계입니다.
- 월요일_평균    해당 역의 승하차 구분별 월요일 평균입니다.
- 화요일_평균    해당 역의 승하차 구분별 화요일 평균입니다.
- 수요일_평균    해당 역의 승하차 구분별 수요일 평균입니다.
- 목요일_평균    해당 역의 승하차 구분별 목요일 평균입니다.
- 금요일_평균    해당 역의 승하차 구분별 금요일 평균입니다.
- 토요일_평균    해당 역의 승하차 구분별 토요일 평균입니다.
- 일요일_평균    해당 역의 승하차 구분별 일요일 평균입니다.
- 모든요일_평균    해당 역의 승하차 구분별 모든요일 평균입니다.

In [ ]:
station_data.columns

Index(['호선명', '역명', '행정구역', '년월구분', '승하차구분', '월요일_합계', '화요일_합계', '수요일_합계',
       '목요일_합계', '금요일_합계', '토요일_합계', '일요일_합계', '모든요일_합계', '월요일_평균', '화요일_평균',
       '수요일_평균', '목요일_평균', '금요일_평균', '토요일_평균', '일요일_평균', '모든요일_평균'],
      dtype='object')

 ### station_data 중에서 '총승하차' 부분이 역별 유동인구를 가늠하기에 적절하다고 판단하여 그 부분을 살펴봅니다.

In [ ]:
# 총승하차만 남김.
station_data = station_data[station_data['승하차구분'] == '총승하차']   

In [ ]:
# 인덱스 재설정
station_data = station_data.reset_index(drop=True)

In [ ]:
#알아보기 쉽게 나머지 불필요한 칼럼들은 제거합니다
station_list = ['호선명','역명','승하차구분','모든요일_합계']
station_data = station_data[station_list]
station_data

,호선명,역명,승하차구분,모든요일_합계
0,3호선,지축,총승하차,1012460
1,경의선,화전,총승하차,1799161
2,경의선,강매,총승하차,1859720
3,경의선,행신,총승하차,5568132
4,경의선,능곡,총승하차,2637855
5,경의선,곡산,총승하차,404413
6,경의선,백마,총승하차,4733842
7,경의선,풍산,총승하차,3577046
8,경의선,일산,총승하차,6808074
9,경의선,탄현,총승하차,5356327


In [ ]:
# station_data 파일에 각 역의 위치를 입히기 시작
station_location

,rail_nm,station_nm,lot_num_addr,road_nm_addr,X,Y
0,3호선,대곡,경기도 고양시 덕양구 대장동 420-1,경기도 고양시 덕양구 대주로107번길 71-81,126.809614,37.632306
1,3호선,대화,경기도 고양시 일산서구 대화동 2221,경기도 고양시 일산서구 중앙로 지하 1569,126.747206,37.675846
2,경의중앙,일산,경기도 고양시 일산서구 일산동 655-691,경기도 고양시 일산서구 경의로 672,126.770067,37.682007
3,3호선,백석,경기도 고양시 일산동구 백석동 1538,경기도 고양시 일산동구 중앙로 지하 1042,126.788148,37.642953
4,경의중앙,대곡,경기도 고양시 덕양구 대장동 420-1,경기도 고양시 덕양구 대주로 107번길 71-81,126.811047,37.631615
5,경의중앙,강매,경기도 고양시 덕양구 행신동 1115-1,경기도 고양시 덕양구 소원로 202,126.844436,37.612290
6,경의중앙,백마,경기도 고양시 일산동구 백석동 92,경기도 고양시 일산동구 경의로 318,126.794067,37.658431
7,경의중앙,능곡,경기도 고양시 덕양구 토당동 454-3,경기도 고양시 덕양구 토당로 35,126.820838,37.618765
8,경의중앙,풍산,경기도 고양시 일산동구 풍동 1042,경기도 고양시 일산동구 경의로 486,126.786528,37.671815
9,경의중앙,행신,경기도 고양시 덕양구 행신동 812,경기도 고양시 덕양구 소원로 114,126.834117,37.612170


In [ ]:
# 호선이 여러개가 지나는 경우, 중복된 값이 있을 수 있다. 
for i in range(len(station_data)) :
    for j in range(len(station_data)) :
        
        if i!=j and station_data['역명'].iloc[i] == station_data['역명'].iloc[j] :
            print(i, j)
            
station_data['모든요일_합계'].iloc[0] = station_data['모든요일_합계'].iloc[0] + station_data['모든요일_합계'].iloc[11]
            
# 지축역은 호선이 중복되어 여러 값이 있음. 따라서 중복값 제거.
station_data = station_data.drop(11)     # 여러번돌리지 않아야 함! 
station_data = station_data.reset_index(drop=True)
station_data

0 11
11 0


,호선명,역명,승하차구분,모든요일_합계
0,3호선,지축,총승하차,1182838
1,경의선,화전,총승하차,1799161
2,경의선,강매,총승하차,1859720
3,경의선,행신,총승하차,5568132
4,경의선,능곡,총승하차,2637855
5,경의선,곡산,총승하차,404413
6,경의선,백마,총승하차,4733842
7,경의선,풍산,총승하차,3577046
8,경의선,일산,총승하차,6808074
9,경의선,탄현,총승하차,5356327


In [ ]:
station_data['X'] = 0
station_data['Y'] = 0

In [ ]:
station_data

,호선명,역명,승하차구분,모든요일_합계,X,Y
0,3호선,지축,총승하차,1182838,0,0
1,경의선,화전,총승하차,1799161,0,0
2,경의선,강매,총승하차,1859720,0,0
3,경의선,행신,총승하차,5568132,0,0
4,경의선,능곡,총승하차,2637855,0,0
5,경의선,곡산,총승하차,404413,0,0
6,경의선,백마,총승하차,4733842,0,0
7,경의선,풍산,총승하차,3577046,0,0
8,경의선,일산,총승하차,6808074,0,0
9,경의선,탄현,총승하차,5356327,0,0


In [ ]:
# 이제 위치를 입히기
for i in range(len(station_data['역명'])):
    for j in range(len(station_location['station_nm'])):
        if station_data['역명'].iloc[i] == station_location['station_nm'].iloc[j] :
            print(i, station_data['역명'].iloc[i], station_location['X'].iloc[j], station_location['Y'].iloc[j])
            station_data['X'].iloc[i] = station_location['X'].iloc[j]
            station_data['Y'].iloc[i] = station_location['Y'].iloc[j]
            break

for i in range(len(station_data)) :
    if station_data['X'].iloc[i] == 0 :
        print('위치정보 못 받은 역은 ', station_data['역명'].iloc[i])


for i in range(len(station_location)) :
    if '지축' in station_location['station_nm'].iloc[i] or '화전' in station_location['station_nm'].iloc[i] :
        print('station_location 파일에서 ',i, station_location['station_nm'].iloc[i])
        
        
        
        

station_data['X'].iloc[0] = station_location['X'].iloc[20]
station_data['Y'].iloc[0] = station_location['Y'].iloc[20]
     
station_data['X'].iloc[1] = station_location['X'].iloc[10]
station_data['Y'].iloc[1] = station_location['Y'].iloc[10]

station_data  # 이제 위치는 다 뽑고, 정렬만 다시 하면 된다

2 강매 126.84443637 37.61228963
3 행신 126.834117 37.61217
4 능곡 126.82083836 37.61876543
5 곡산 126.80151439 37.64612663
6 백마 126.794067 37.658431
7 풍산 126.786528 37.671815
8 일산 126.77006749 37.68200683
9 탄현 126.76116553 37.69385707
10 원흥 126.87260033 37.65074577
11 삼송 126.89558955 37.65310988
12 원당 126.84303034 37.65331645
13 화정 126.83266635 37.63459661
14 대곡 126.809614 37.632306
15 백석 126.7881483 37.64295283
16 마두 126.77764972 37.65217351
17 정발산 126.77335864 37.65949317
18 주엽 126.76122972 37.67014215
19 대화 126.747206 37.675846
위치정보 못 받은 역은  지축
위치정보 못 받은 역은  화전
station_location 파일에서  10 화전(한국항공대)
station_location 파일에서  20 지축역


/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,호선명,역명,승하차구분,모든요일_합계,X,Y
0,3호선,지축,총승하차,1182838,126.913728,37.648118
1,경의선,화전,총승하차,1799161,126.867947,37.603298
2,경의선,강매,총승하차,1859720,126.844436,37.612290
3,경의선,행신,총승하차,5568132,126.834117,37.612170
4,경의선,능곡,총승하차,2637855,126.820838,37.618765
5,경의선,곡산,총승하차,404413,126.801514,37.646127
6,경의선,백마,총승하차,4733842,126.794067,37.658431
7,경의선,풍산,총승하차,3577046,126.786528,37.671815
8,경의선,일산,총승하차,6808074,126.770067,37.682007
9,경의선,탄현,총승하차,5356327,126.761166,37.693857


In [ ]:
# 사람들이 많이 다니는 순으로 정렬
station_data = station_data.sort_values(by='모든요일_합계',ascending=False)  
station_data

#다시 보기좋게 인덱스 정렬
station_data = station_data.reset_index(drop=True)   
station_data

# 저장해놓기
station_data1 = station_data.copy()   
station_data = station_data1

# 완료
station_data    


,호선명,역명,승하차구분,모든요일_합계,X,Y
0,일산선,화정,총승하차,14522105,126.832666,37.634597
1,일산선,삼송,총승하차,11526325,126.895590,37.653110
2,일산선,대화,총승하차,10008025,126.747206,37.675846
3,일산선,원당,총승하차,8607849,126.843030,37.653316
4,일산선,백석,총승하차,8533226,126.788148,37.642953
5,일산선,주엽,총승하차,7234466,126.761230,37.670142
6,일산선,정발산,총승하차,6843625,126.773359,37.659493
7,경의선,일산,총승하차,6808074,126.770067,37.682007
8,일산선,마두,총승하차,6222179,126.777650,37.652174
9,경원선,원흥,총승하차,5757420,126.872600,37.650746


# 버스데이터를 기반으로 인구유동성 데이터 추출하기 

## 데이터 설명 
 ### 20.고양시_버스정류소
 - 현재 기준의 고양시 버스 정류소, 마을버스 정류소 위치 정보입니다.
- STATION_NM    버스정류장 명 입니다.
- STATION_ID    버스정류장 고유 ID 입니다.
- X    버스정류장의 위도 입니다.
- Y    버스정류장의 경도 입니다.

 ### 21.버스_정류장별_승하차_정보
 - 최근 1년치 고양시 버스 정류장별 승하차 정보입니다.
- STATION_ID    버스정류장 고유 ID 입니다.
- STATION_NM    버스정류장 명 입니다.
- GETON_CNT    버스정류장 승차인원 수 입니다.

### 두 파일을 병합하기 
- STATION_ID: 정류장ID
- STATION_NM: 정류장 이름
- GETON_CNT: 승하차인원수
- X: 경도
- Y: 위도

In [ ]:
bustatonf = pd.read_csv(input_path.joinpath('20.고양시_버스정류소.csv'))
bustat = pd.read_csv(input_path.joinpath('21.버스_정류장별_승하차_정보.csv'))

In [ ]:
# merge 이용
bus = bustat.merge(bustatonf, on=['STATION_NM','STATION_ID'])
bus

#2823개

,STATION_ID,STATION_NM,GETON_CNT,X,Y
0,218001046,화정역3호선,229091,126.832410,37.635347
1,218001083,원당역,219186,126.843885,37.653079
2,218000974,화정역3호선,197084,126.832827,37.634486
3,218000974,화정역3호선,5335,126.832827,37.634486
4,219000193,대화역,191696,126.747539,37.676390
...,...,...,...,...,...
2818,219000882,환경사업소,0,126.726069,37.659134
2819,219000968,장항굴다리,0,126.761935,37.647081
2820,218000827,새벽교회,0,126.858807,37.721591
2821,218000888,남양유업,0,126.852764,37.666689


## groupby를 통해 동명의 정류장끼리 묶는다
 - 위치가 조금씩 다르지만 같은 한글이름을 사용하는 정류장들을 묶음

In [ ]:
bus_grouped = bus.groupby(['STATION_NM','STATION_ID','GETON_CNT'],as_index=False).sum()
bus_grouped

,STATION_NM,STATION_ID,GETON_CNT,X,Y
0,(구)효자동주민센터,218000258,149,126.919747,37.652634
1,(구)효자동주민센터,218000258,195,126.919747,37.652634
2,(구)효자동주민센터,218000265,19,126.919892,37.652603
3,(구)효자동주민센터,218000265,27,126.919892,37.652603
4,16단지.삼송초교,218001182,9686,126.882791,37.653627
...,...,...,...,...,...
2818,흰돌마을(중),219000453,6900,126.781373,37.648172
2819,흰돌마을(중),219000453,7817,126.781373,37.648172
2820,흰돌마을.서안아파트,219000161,934,126.785637,37.641313
2821,흰돌마을.서안아파트,219000161,10467,126.785637,37.641313


In [ ]:
# 재정의
bus2 = bus_grouped

## 동명의 정류장 중 경위도값이 같은 정류장끼리는 GETON_CNT를 더해준다 (groupby)

In [ ]:
# 위치가 같은 정류소 중복을 방지
bus3 = bus2.groupby(['STATION_NM','STATION_ID','X','Y'], as_index=False).sum()
bus3

,STATION_NM,STATION_ID,X,Y,GETON_CNT
0,(구)효자동주민센터,218000258,126.919747,37.652634,344
1,(구)효자동주민센터,218000265,126.919892,37.652603,46
2,16단지.삼송초교,218001182,126.882791,37.653627,9686
3,16단지.삼송초교,218001187,126.883295,37.653999,1126
4,1군단앞,218000498,126.911514,37.716309,21
...,...,...,...,...,...
2076,희망원,219000229,126.821292,37.702977,1248
2077,흰돌5단지.서안아파트,219000213,126.785729,37.640872,14603
2078,흰돌마을(중),219000453,126.781373,37.648172,14717
2079,흰돌마을.서안아파트,219000161,126.785637,37.641313,11401


In [ ]:
bus3.head(10)

,STATION_NM,STATION_ID,X,Y,GETON_CNT
0,(구)효자동주민센터,218000258,126.919747,37.652634,344
1,(구)효자동주민센터,218000265,126.919892,37.652603,46
2,16단지.삼송초교,218001182,126.882791,37.653627,9686
3,16단지.삼송초교,218001187,126.883295,37.653999,1126
4,1군단앞,218000498,126.911514,37.716309,21
5,1군단앞,218000500,126.912488,37.717179,321
6,2단지관리사무실앞,218000743,126.836772,37.646522,303
7,2단지라이프아파트앞,218000932,126.837170,37.645662,24
8,2단지라이프아파트앞,218001009,126.837020,37.645986,14376
9,5단지관리사무실앞,218000740,126.836483,37.642546,7119


## 상위 몇개만 뽑아보면 버스승하차 횟수가 많은 지역을 알 수 있을 것이다

In [ ]:
bus_a = bus3.sort_values(by='GETON_CNT', ascending=False)
bus_a

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
1340,원당역,218001083,126.843885,37.653079,219186
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
...,...,...,...,...,...
1325,원골마을회관,218000597,126.864203,37.581361,0
888,새벽교회,218000827,126.858807,37.721591,0
398,덕은동,218000594,126.860548,37.581514,0
56,가좌동성당,219001111,126.726562,37.697732,0


In [ ]:
# 전체 버스 정류장 평균 승하차인원이 4700여명 정도 된다
bus_a.describe() 

,STATION_ID,X,Y,GETON_CNT
count,2.081000e+03,2081.000000,2081.000000,2081.000000
mean,2.185258e+08,126.819148,37.666634,4688.834695
std,1.033896e+06,0.053995,0.032485,16699.517366
min,2.180000e+08,126.679997,37.581102,0.000000
25%,2.180008e+08,126.777363,37.644999,70.000000
50%,2.180013e+08,126.823145,37.668142,573.000000
75%,2.190006e+08,126.859858,37.691194,3677.000000
max,2.415011e+08,126.962809,37.747089,361686.000000


## 승하차별 상위 n개의 버스데이터를 만들어 살펴보자 
 - csv파일로 만들어 저장한 뒤, qgis프로그램을 이용하여 지도에 좌표를 찍는 시각화를 통해서 한 눈에 알아볼 수 있음

In [ ]:
# 이용이 많은 버스정류장 상위 30개를 살펴볼 수 있다
bus_a_top30 = bus_a.head(30)

# 이용이 많은 버스정류장 상위 50개를 살펴볼 수 있다
bus_a_top50 = bus_a.head(50)

# 이용이 많은 버스정류장 상위 100개를 살펴볼 수 있다
bus_a_top100 = bus_a.head(100)

In [ ]:
# csv파일로 만들어 추후 qgis 시각화에 이용
bus_a_top30.to_csv('bus_a_top30.csv')
bus_a_top50.to_csv('bus_a_top50.csv')
bus_a_top100.to_csv('bus_a_top100.csv')

In [ ]:
# 재정의
bus_a_top100 = bus_a_top100.copy()
bus_a_top100

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
1340,원당역,218001083,126.843885,37.653079,219186
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
...,...,...,...,...,...
899,샘터마을2단지.신능중학교,218000118,126.833738,37.621644,19139
1916,행신역,218001019,126.835049,37.612941,18961
1087,숲속마을4.5단지,219000460,126.800155,37.669089,18927
1745,탄현효성주은아파트,219000315,126.768340,37.690707,18923


- 중복되는 정류장 (= 상/하행이 있는 정류장)과 중복되지 않는 정류장 분류
- 중복되는 정류장 -> 어쨌든 양방향 유동인구가 많다는 것을 의미 -->  합쳐진 데이터로 분석


In [ ]:
# 여기서는 중복되지 않는 정류장 이름들을 살펴본다

bus_a_top100_no_dup = bus_a_top100.drop_duplicates(["STATION_NM"], keep=False)
bus_a_top100_no_dup.info()

# # 총59곳 --> 중복되는 것의 짝이 두개씩이면 중복되지 않는 것도 짝수 값이 나올텐데
# 홀수값 --> 정류장이름이 중복되는 것이 세개씩 있는 것도 있을 것이라고 추측가능함.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 1340 to 442
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION_NM  59 non-null     object 
 1   STATION_ID  59 non-null     int64  
 2   X           59 non-null     float64
 3   Y           59 non-null     float64
 4   GETON_CNT   59 non-null     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 2.8+ KB


In [ ]:
# 중복되지 않는 정류장
bus_a_top100_no_dup

,STATION_NM,STATION_ID,X,Y,GETON_CNT
1340,원당역,218001083,126.843885,37.653079,219186
855,삼송역8번출구,218001200,126.896880,37.653835,102780
853,삼송역5번출구,218000284,126.893512,37.653499,82318
2020,화정역.롯데마트,218000019,126.830748,37.632918,66042
851,삼송역3번출구,218001280,126.895430,37.652687,58272
854,삼송역6번출구,218000627,126.896329,37.653501,55333
852,삼송역4번출구,218000090,126.893807,37.653250,55069
1367,원흥역3번출구,218000696,126.873272,37.650091,50388
2013,화정1동행정복지센터,218000102,126.834066,37.646016,46277
1947,현대백화점,219000625,126.752013,37.668475,46162


In [ ]:
# 중복되는 정류소 뽑기

bus_top_data = pd.concat([bus_a_top100, bus_a_top100_no_dup, bus_a_top100_no_dup]).drop_duplicates(keep=False)
bus_top_data

# "리스쇼핑.원당시장"은 값이 3개다
# 왜인가 했더니 (지역의 특성상) 로타리라서 정류장이 많음 (총 4개)
# 이 곳은 양방향 유동인구가 많다고 할 수 있겠다

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
534,마두역(중),219000370,126.777480,37.652390,162142
1478,일산동구청(중),219000356,126.774287,37.657816,156731
1590,주엽역(중),219000189,126.761634,37.670067,154395
1477,일산동구청(중),219000191,126.774728,37.657438,151423
535,마두역(중),219000383,126.778044,37.651717,148257
1591,주엽역(중),219000363,126.760893,37.670280,119326


In [ ]:
# 총41곳
bus_top_data['STATION_NM'].count()

41

In [ ]:
# 짝이 다 맞는지 확인하기. 완료
bus_top_data.groupby('STATION_NM').count()

,STATION_ID,X,Y,GETON_CNT
STATION_NM,,,,
강선마을(중),2,2,2,2
고양경찰서(중),2,2,2,2
고양동시장,2,2,2,2
관산동.벽제시장,2,2,2,2
대화역,2,2,2,2
리스쇼핑.원당시장,3,3,3,3
마두1동행정복지센터,2,2,2,2
마두역(중),2,2,2,2
문촌마을(중),2,2,2,2
